In [1]:
import sim as vrep
import numpy as np
from numpy import linalg
import time
import math
import sys

from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi
from time import sleep


In [2]:
#!/usr/bin/env python
from __future__ import division
import numpy as np
from scipy.linalg import expm


tstep = 5 
jointNum = 6
baseName = 'UR3'
jointName = 'UR3_joint'


"""
Use 'expm' for matrix exponential.
Angles are in radian, distance are in meters.
"""

def Get_MS():
	# =================== Your code starts here ====================#
	# Fill in the correct values for S1~6, as well as the M matrix
	M = np.eye(4)
	S = np.zeros((6,6))
	
	# find M and S by measuring the UR3 robot arm
	M = np.array([[1.,0.,0.,-0.38113],[0.,1.,0.,8.5324e-5],[0.,0.,1.,0.6511], [0.,0.,0.,1.]])
	S = np.array([[0.,-1.,-1.,-1.,0.,-1.], [0.,0.,0.,0.,0.,0.],[1.,0.,0.,0.,1.,0.],[8.5577e-05,0.,0.,0.,8.513e-05,0.],[-0.20012,-0.60887,-0.85252,-1.0658,-0.08775,-1.1511],[0,5.4419e-05,0.0001303,8.5157e-05,0,8.5086e-05]])

	
	# ==============================================================#
	return M, S



"""
Function that calculates encoder numbers for each motor
"""
def lab_fk(theta1, theta2, theta3, theta4, theta5, theta6):

	# Initialize the return_value 
	return_value = [None, None, None, None, None, None]

	# =========== Implement joint angle to encoder expressions here ===========
	print("Foward kinematics calculated:\n")

	# =================== Your code starts here ====================#
	theta = np.array([theta1,theta2,theta3,theta4,theta5,theta6])
	T = np.eye(4)

	M, S = Get_MS()

	s1_mat = np.array([[0, -S[2][0], S[1][0], S[3][0]],[S[2][0], 0, -S[0][0], S[4][0]], [-S[1][0], S[0][0], 0, S[5][0]], [0,0,0,0]])
	s2_mat = np.array([[0, -S[2][1], S[1][1], S[3][1]],[S[2][1], 0, -S[0][1], S[4][1]], [-S[1][1], S[0][1], 0, S[5][1]], [0,0,0,0]])
	s3_mat = np.array([[0, -S[2][2], S[1][2], S[3][2]],[S[2][2], 0, -S[0][2], S[4][2]], [-S[1][2], S[0][2], 0, S[5][2]], [0,0,0,0]])
	s4_mat = np.array([[0, -S[2][3], S[1][3], S[3][3]],[S[2][3], 0, -S[0][3], S[4][3]], [-S[1][3], S[0][3], 0, S[5][3]], [0,0,0,0]])
	s5_mat = np.array([[0, -S[2][4], S[1][4], S[3][4]],[S[2][4], 0, -S[0][4], S[4][4]], [-S[1][4], S[0][4], 0, S[5][4]], [0,0,0,0]])
	s6_mat = np.array([[0, -S[2][5], S[1][5], S[3][5]],[S[2][5], 0, -S[0][5], S[4][5]], [-S[1][5], S[0][5], 0, S[5][5]], [0,0,0,0]])

	T = expm(s1_mat*theta1).dot(expm(s2_mat*theta2)).dot(expm(s3_mat*theta3)).dot(expm(s4_mat*theta4)).dot(expm(s5_mat*theta5)).dot(expm(s6_mat*theta6)).dot(M)

	return T

def quaternion_from_matrix(matrix, isprecise=False):
    """Return quaternion from rotation matrix."""
    M = np.array(matrix, dtype=np.float64, copy=False)[:4, :4]
    if isprecise:
        q = np.empty((4, ))
        t = np.trace(M)
        if t > M[3, 3]:
            q[0] = t
            q[3] = M[1, 0] - M[0, 1]
            q[2] = M[0, 2] - M[2, 0]
            q[1] = M[2, 1] - M[1, 2]
        else:
            i, j, k = 0, 1, 2
            if M[1, 1] > M[0, 0]:
                i, j, k = 1, 2, 0
            if M[2, 2] > M[i, i]:
                i, j, k = 2, 0, 1
            t = M[i, i] - (M[j, j] + M[k, k]) + M[3, 3]
            q[i] = t
            q[j] = M[i, j] + M[j, i]
            q[k] = M[k, i] + M[i, k]
            q[3] = M[k, j] - M[j, k]
            q = q[[3, 0, 1, 2]]
        q *= 0.5 / math.sqrt(t * M[3, 3])
    else:
        m00 = M[0, 0]
        m01 = M[0, 1]
        m02 = M[0, 2]
        m10 = M[1, 0]
        m11 = M[1, 1]
        m12 = M[1, 2]
        m20 = M[2, 0]
        m21 = M[2, 1]
        m22 = M[2, 2]
        # symmetric matrix K
        K = np.array([[m00-m11-m22, 0.0,         0.0,         0.0],
                         [m01+m10,     m11-m00-m22, 0.0,         0.0],
                         [m02+m20,     m12+m21,     m22-m00-m11, 0.0],
                         [m21-m12,     m02-m20,     m10-m01,     m00+m11+m22]])
        K /= 3.0
        # quaternion is eigenvector of K that corresponds to largest eigenvalue
        w, V = np.linalg.eigh(K)
        q = V[[3, 0, 1, 2], np.argmax(w)]
    if q[0] < 0.0:
        np.negative(q, q)
    return q

In [3]:

print ('UR3 MOVER started')
vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to V-REP
if clientID != -1:
    print ('Connected to UR3 Simulator')

    # Make a Dummy
    errorCode, dummy_handle_0 = vrep.simxCreateDummy(clientID,0.1,None,vrep.simx_opmode_oneshot_wait)

        
        # theta_list[i] = (theta_list[i] * math.pi) / 180

    # List to store our joint handles in
    joint_handles = []

    # Loop through all the robot's joints and get unique ID's for each
    for i in range(6):
        joint_name = 'UR3_joint' + str(i + 1)
        errorCode, handle = vrep.simxGetObjectHandle(clientID, joint_name, vrep.simx_opmode_oneshot_wait)
        joint_handles.append(handle)
    _, baseHandle = vrep.simxGetObjectHandle(clientID, baseName, vrep.simx_opmode_blocking)
    #Set to dummy to tool end
    vrep.simxSetObjectPosition(clientID, dummy_handle_0, joint_handles[5], (0,0,0), 
                               vrep.simx_opmode_oneshot_wait)
    vrep.simxSetObjectOrientation(clientID,dummy_handle_0,-1,(0,0,0),vrep.simx_opmode_oneshot_wait)

    errorCode, pos = vrep.simxGetObjectPosition(clientID,joint_handles[5],-1,vrep.simx_opmode_oneshot_wait)
    print("Initial end pos:",pos)
        # Joint:          1  2  3  4  5  6  
    theta_list = np.array([1.0,1.57,1.0,1.57,0.0,1.57])

    for i in range(6):
        theta_list[i] = float(input("Enter number for joint {}: ".format(i+1)))
    # Show our predicted end effector position
    pose = lab_fk(theta_list[0],theta_list[1],theta_list[2],theta_list[3],theta_list[4],theta_list[5])
    print("homogeneous transformation matrix T:\n",pose)

    input("Press any key to move to predicted location")

    # Set the position of each joint
    for i in range (6):
        vrep.simxSetJointTargetPosition(clientID,joint_handles[i],theta_list[i], vrep.simx_opmode_oneshot_wait)
        # print("Setting joint", i+1, "to", theta_list[i])

    sleep(2)

    errorCode, dummy_handle_3 = vrep.simxCreateDummy(clientID, 0.1, None, vrep.simx_opmode_oneshot_wait)
    vrep.simxSetObjectPosition(clientID,dummy_handle_3,joint_handles[5],(0,0,0),vrep.simx_opmode_oneshot_wait)
    vrep.simxSetObjectOrientation(clientID,dummy_handle_3,joint_handles[5],(0,0,0),vrep.simx_opmode_oneshot_wait)

    errorCode, angles = vrep.simxGetObjectQuaternion(clientID,joint_handles[5], baseHandle,vrep.simx_opmode_oneshot_wait)
    print("Actual qua:",angles)
    errorCode, pos = vrep.simxGetObjectPosition(clientID,joint_handles[5], baseHandle,vrep.simx_opmode_oneshot_wait)
    print("Actual end pos:",pos)
#     for i in range(6):
#         theta = vrep.simxGetJointPosition(clientID, joint_handles[i], vrep.simx_opmode_oneshot_wait)
#         print("Theta",i,"is",theta)

    # sleep(10)
    input("Press any key to revert to origin...")
    print("")

    for i in range (6):
        vrep.simxSetJointTargetPosition(clientID,joint_handles[i],0,vrep.simx_opmode_oneshot_wait)
        print("Setting joint", i+1, "back to 0")

    sleep(0.5)

    vrep.simxRemoveObject(clientID,dummy_handle_0,vrep.simx_opmode_oneshot_wait)
    vrep.simxRemoveObject(clientID,dummy_handle_3,vrep.simx_opmode_oneshot_wait)

    # Now close the connection to V-REP:
    vrep.simxFinish(clientID)
else:
    print ('Failed connecting to remote API server')
    sys.exit("Connection Failed")
print ('Program ended')

UR3 MOVER started
Connected to UR3 Simulator
Initial end pos: [-0.36193427443504333, 0.2969452142715454, 0.6940981149673462]
Enter number for joint 1: 1
Enter number for joint 2: 1
Enter number for joint 3: 1
Enter number for joint 4: 1
Enter number for joint 5: 1
Enter number for joint 6: 1
Foward kinematics calculated:

homogeneous transformation matrix T:
 [[ 0.99291395  0.09746501 -0.06798864 -0.638023  ]
 [ 0.0045499  -0.60288387 -0.79781598  0.52438207]
 [-0.11874839  0.79185328 -0.59905526  0.92249627]
 [ 0.          0.          0.          1.        ]]
Press any key to move to predicted location1
Actual qua: [0.4873116910457611, -0.44649678468704224, -0.5507962107658386, 0.509697437286377]
Actual end pos: [-0.4066847562789917, 0.12821471691131592, 0.06834452599287033]
Press any key to revert to origin...1

Setting joint 1 back to 0
Setting joint 2 back to 0
Setting joint 3 back to 0
Setting joint 4 back to 0
Setting joint 5 back to 0
Setting joint 6 back to 0


NameError: name 'dummy_handle_1' is not defined